# Flashback HFT Backtesting Engine - Quickstart

This notebook demonstrates a complete backtesting workflow using the Flashback HFT backtesting engine.

## What we'll do:
1. Generate synthetic market data
2. Run a momentum imbalance strategy
3. Analyze performance metrics
4. Visualize results


In [ ]:
import sys
import os
sys.path.append('../../')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

# Flashback imports
from flashback.cli.runner import BacktestRunner
from flashback.config.parser import load_config
from flashback.metrics.performance import PerformanceAnalyzer

# Set up plotting
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

print("✅ Imports successful!")


## 1. Generate Synthetic Market Data

First, let's generate realistic market data with microprice movements, bursts, and order flow imbalance.


In [ ]:
# Generate synthetic data
from examples.generate_synthetic import generate_synthetic_data

# Generate 200k events for quick demo
book_file, trade_file = generate_synthetic_data(
    symbol="AAPL",
    num_events=200000,
    seed=42,
    output_dir="../../examples/sample_data"
)

print(f"📊 Generated data files:")
print(f"   Book: {book_file}")
print(f"   Trades: {trade_file}")


## 2. Run Backtest with Momentum Imbalance Strategy

Now let's run a backtest using the momentum imbalance strategy on our synthetic data.


In [ ]:
# Load backtest configuration
config_path = "../../config/backtest.yaml"
config = load_config(config_path)

# Update config to use our generated data
config.data.path = book_file
config.report.output_dir = "../../runs/quickstart_demo"

print("⚙️  Backtest Configuration:")
print(f"   Strategy: {config.strategy.name}")
print(f"   Data: {config.data.path}")
print(f"   Output: {config.report.output_dir}")

# Run the backtest
print("🚀 Starting backtest...")

runner = BacktestRunner(config)
results = runner.run()

print("✅ Backtest completed!")
print(f"📊 Results saved to: {config.report.output_dir}")


## 3. Analyze Performance Results

Let's examine the performance metrics and trading results.


In [ ]:
# Load performance results
results_dir = Path(config.report.output_dir)
performance_file = results_dir / "performance.json"

if performance_file.exists():
    import json
    with open(performance_file, 'r') as f:
        performance = json.load(f)
    
    print("📈 Performance Metrics:")
    print(f"   Total Return: {performance.get('total_return', 0):.2%}")
    print(f"   Sharpe Ratio: {performance.get('sharpe_ratio', 0):.2f}")
    print(f"   Max Drawdown: {performance.get('max_drawdown', 0):.2%}")
    print(f"   Total Trades: {performance.get('total_trades', 0)}")
    print(f"   Hit Rate: {performance.get('hit_rate', 0):.2%}")
    print(f"   Turnover: {performance.get('turnover', 0):.2f}")
else:
    print("❌ Performance file not found")

print("🎉 Quickstart completed successfully!")
print(f"📁 All results saved to: {config.report.output_dir}")
